In this Jupyter Notebook, we will be re-writing the implementation of Uniswap V3 in Python.

In [2]:
import numpy as np
import pandas as pd
import math
from sympy import symbols, Symbol, Eq, nsolve, solve, sqrt

from decimal import Decimal, getcontext
getcontext().prec = 50

# Variables

Here, we define our variables.

In [3]:
"""
The current_state variable is a dictionary that maps Initiated ticks to another dictionary, which gives properties of the liquidity pool at that tick:
1) Net liquidity at each tick
2) Gross liquidity at each tick

An example will be:
{'tick = 1':{'net_liquidity':1, 'gross_liquidity':1},
 'tick = 3':{'net_liquidity':3, 'gross_liquidity':3}}
"""

current_state = {}

"""
The current_spot_price is a variable that gives the current swap rate of the pool (in terms of token1/token0)
"""

current_spot_price = 0

"""
The lp_dataframe stores a record of how much liquidity and at what range liquidity providers current have in the pool
"""

lp_dataframe = pd.DataFrame(columns = ['address','tick_lower','tick_upper','L'])

"""
The fees_collected variable is a dictionary that keeps track of the fee earned by every LPs. Key = Address, Value = [Token 0 earned, Token 1 earned]
A new key is appended to the dictionary whenever a new LP provides liquidity.
"""

fees_collected = {}

# Functions

Here, we write functions that will be useful. All functions are written in relation to the UniSwap V3 protocol.

### Get Tick

This returns the tick that corresponds to a given price.

In [4]:
"""
Parameters:
_price_ (float): this is the price

Results:
tick (int): this is the tick that corresponds to the given price
"""

def Get_Tick(_price_):
    return math.floor(math.log(10**12/_price_,1.0001))

### Get Price

This returns the price that corresponds to a given tick.

In [5]:
"""
Parameters:
_tick_ (int): this is the tick

Results:
price (float): this is the price that corresponds to the given tick

"""

def Get_Price(_tick_):
    return Decimal(1/(1.0001**_tick_*10**-12))

### Get Total Liquidity

This returns the total liquidity of the pool at any ticks.

In [6]:
"""
Parameters:
_current_state_ (dict): this gives the current state of the pool
_tick_ (int): this is the tick

Results:
total_liquidity (float): this gives total liquidity available at the given tick
"""

def Get_Total_Liquidity(_current_state_, _tick_):

    # Get all initiated ticks less than _tick_
    all_ticks_before = [ticks for ticks in list(_current_state_.keys()) if ticks <= _tick_]

    # Run a loop to sum all the net liquidity to get total liquidity
    total_liquidity = 0

    for ticks in all_ticks_before:
        total_liquidity = total_liquidity + _current_state_[ticks]['net_liquidity']
    
    return total_liquidity

### Get Current Spot Price

This function gives the liquidity and current spot price of the liquidity pool given inputs of liquidity provision/withdrawal.

In [7]:
def Get_Current_Spot_Price(_token0_amount_, _token1_amount_, _price_lower_, _price_upper_, _current_spot_price_):

    if _token0_amount_ > 0 and _token1_amount_ > 0:

        L,P = symbols('L,P')
        eq1 = Eq(L*(1/sqrt(P)-1/sqrt(_price_upper_)), _token0_amount_)
        eq2 = Eq(L*(sqrt(P) - sqrt(_price_lower_)), _token1_amount_)
        results = nsolve([eq1,eq2], [L,P], [_price_lower_*_price_upper_, _current_spot_price_], prec = 50)
        L = results[0]
        P = results[1]

    else: 

        L = Symbol('L')
        L = solve((_token0_amount_ + L/(_price_upper_.sqrt())) * (_token1_amount_ + L*(_price_lower_.sqrt())) - L**2, L)
        L = L[1]
        P = _current_spot_price_

    return L, P

### Provide Liquidity

This function is used to add liquidity to the liquidity pool.

In [8]:
def Provide_Liquidity(_current_state_, _current_spot_price_, _lp_dataframe_, _fees_collected_, _address_, _tick_lower_, _tick_upper_, _token0_amount_, _token1_amount_):

    """
    Initial Calculations
    """
    # First, get prices that corresponds to _tick_lower_ and _tick_upper_
    price_lower = Get_Price(_tick_upper_)
    price_upper = Get_Price(_tick_lower_)

    """
    Liquidity Provision
    """
    # For liquidity provision, we split into 2 cases:

    # Case 1) Liquidity pool is empty
    if not bool(_current_state_.keys()):

        # Get total liquidity from this provision, solve for L
        L, new_spot_price = Get_Current_Spot_Price(_token0_amount_, _token1_amount_, price_lower, price_upper, (price_lower+price_upper)/2)
        
        _current_state_[_tick_lower_] = {'net_liquidity':L, 'gross_liquidity':L}
        _current_state_[_tick_upper_] = {'net_liquidity':-L, 'gross_liquidity':L}
        _lp_dataframe_.loc[len(_lp_dataframe_)] = [_address_, _tick_lower_, _tick_upper_, L]
        _fees_collected_[_address_] = [0,0]

        print("System Message: Trader sends", str(_token0_amount_), "token0 and", str(_token1_amount_), "token1 into the pool")

        # Return current_state and all function below do not need to be run
        return _current_state_, new_spot_price, _lp_dataframe_, _fees_collected_

    """
    Initial Condition Check
    """
    # Check whether deposition amount is within valid range:
    if _current_spot_price_ < price_lower and _token1_amount_ > 0:

        raise TypeError("Invalid token deposit, USDC needs to be 0")
    
    elif _current_spot_price_ > price_upper and _token0_amount_ > 0:

        raise TypeError("Invalid token deposit, WETH needs to be 0")
    
    else:

        # First, solve for L
        L, new_spot_price = Get_Current_Spot_Price(_token0_amount_, _token1_amount_, price_lower, price_upper, _current_spot_price_)

        # Liquidity provision should not change the spot price
        if np.abs(new_spot_price/_current_spot_price_ - 1) > 1e-5 and price_lower < _current_spot_price_ < price_upper:   
            raise TypeError("Invalid token deposit, too far from current spot price")
        

        # Case 2) Liquidity pool is not empty
        
        # If tick_lower provided is not already initiated, we initiate the tick_lower
        if _tick_lower_ not in _current_state_.keys():

            # Add the new initiated tick and corresponding information
            _current_state_[_tick_lower_] = {'net_liquidity':L, 'gross_liquidity':L}

            # Sort dictionary by keys
            _current_state_ = dict(sorted(_current_state_.items()))

        # If tick_lower is already initiated, we can simply sum the net liquidity, gross liquidity and net real reserves
        else:

            _current_state_[_tick_lower_]['net_liquidity'] = _current_state_[_tick_lower_]['net_liquidity'] + L
            _current_state_[_tick_lower_]['gross_liquidity'] = _current_state_[_tick_lower_]['gross_liquidity'] + L


        # If tick_upper provided is not already initiated, we initiate the tick_upper
        if _tick_upper_ not in _current_state_.keys():
            
            # Add the new initiated tick and corresponding information
            _current_state_[_tick_upper_] = {'net_liquidity':-L, 'gross_liquidity':L}

            # Sort dictionary by keys
            _current_state_ = dict(sorted(_current_state_.items()))

        # If tick_upper is already initiated, we can simply minus the net liquidity, net real reserves and sum the gross liquidity
        else:    
            _current_state_[_tick_upper_]['net_liquidity'] = _current_state_[_tick_upper_]['net_liquidity'] - L
            _current_state_[_tick_upper_]['gross_liquidity'] = _current_state_[_tick_upper_]['gross_liquidity'] + L
            
        
        # Add address if this address hasn't previously provided liquidity
        if _address_ not in _fees_collected_.keys():
            _fees_collected_[_address_] = [0,0]

        # Lastly, add this liquidity provision to the dataframe
        _lp_dataframe_.loc[len(_lp_dataframe_)] = [_address_, _tick_lower_, _tick_upper_, L]
        _lp_dataframe_ = _lp_dataframe_.groupby(['address','tick_lower','tick_upper']).sum().reset_index()
        # WE DO NOT NEED TO UPDATE THE SPOT PRICE BECAUSE SPOT PRICE SHOULD NOT CHANGE WITH LIQUDIITY PROVISION

        print("System Message: Trader sends", str(_token0_amount_), "token0 and", str(_token1_amount_), "token1 into the pool")

        """
        Returns
        """
        # Return the new _current_state_
        return _current_state_, _current_spot_price_, _lp_dataframe_, _fees_collected_


### Withdraw Liquidity

This function is used to withdraw liquidity from the liquidity pool.

In [9]:
"""
Parameters:
_current_state_ (dict): this gives the current state (before withdrawing liquidity) of the pool
_current_spot_price_ (float): this gives the current spot price of the pool
_lp_dataframe_ (pandas dataframe): this gives the current LPs of the pool
_fees_collected_ (dict): this gives the fees collected by each LP
_address_ (hexbytes): this is the address that provided the liquidity
_tick_lower_ (int): this gives the lower tick for this withdrawal
_tick_upper_ (int): this gives the upper tick for this withdrawal
_token0_amount_ (float): this gives the amount of token 0 withdrawn
_token1_amount_ (float): this gives the amount of token 1 withdrawn

Results:
_current_state_ (dict): this gives the new state (after withdrawing liquidity) of the pool
_current_spot_price_ (float): this gives the new spot price of the pool
_lp_dataframe_ (pandas dataframe): this gives the new LPs of the pool
"""

def Withdraw_Liquidity(_current_state_, _current_spot_price_, _lp_dataframe_, _address_, _tick_lower_, _tick_upper_, _token0_amount_, _token1_amount_):

    """
    Initial Calculations
    """
    # First, get prices that corresponds to _tick_lower_ and _tick_upper_
    price_lower = Get_Price(_tick_upper_)
    price_upper = Get_Price(_tick_lower_)

    """
    Initial Condition Check
    """
    # Check whether withdrawal amount is within valid range:
    if _current_spot_price_ < price_lower and _token1_amount_ > 0:

        raise TypeError("Invalid liquidity withdrawal, USDC needs to be 0")
    
    elif _current_spot_price_ > price_upper and _token0_amount_ > 0:

        raise TypeError("Invalid liquidity withdrawal, WETH needs to be 0")
    
    else:
        
        # First solve for L
        L, new_spot_price = Get_Current_Spot_Price(_token0_amount_, _token1_amount_, price_lower, price_upper, _current_spot_price_)

        # Liquidity provision should not change the spot price
        if np.abs(new_spot_price - _current_spot_price_) > 1e-3 and price_lower < _current_spot_price_ < price_upper:
            raise TypeError("Invalid token withdrawl, too far from current spot price")

        """
        Initial Condition Check
        """
        # Check whether if the LP has provided enough LP for this withdrawal: (we use a buffer here to prevent rounding issues)
        if (_lp_dataframe_[(_lp_dataframe_['address'] == _address_) & (_lp_dataframe_['tick_lower'] == _tick_lower_) & (_lp_dataframe_['tick_upper'] == _tick_upper_)]['L'].values[0] + 1e-3) < L:
            
            raise TypeError("Invalid liquidity withdrawal, you do not have enough liquidity in the pool")
        
        else:
            
            # If difference is below 1e-6, we treat them as the same
            if np.abs(_lp_dataframe_[(_lp_dataframe_['address'] == _address_) & (_lp_dataframe_['tick_lower'] == _tick_lower_) & (_lp_dataframe_['tick_upper'] == _tick_upper_)]['L'].values[0] - L) < 1e-3:
                L = _lp_dataframe_[(_lp_dataframe_['address'] == _address_) & (_lp_dataframe_['tick_lower'] == _tick_lower_) & (_lp_dataframe_['tick_upper'] == _tick_upper_)]['L'].values[0]

            # Update _current_state_
            _current_state_[_tick_lower_]['net_liquidity'] = _current_state_[_tick_lower_]['net_liquidity'] - L
            _current_state_[_tick_lower_]['gross_liquidity'] = _current_state_[_tick_lower_]['gross_liquidity'] - L

            _current_state_[_tick_upper_]['net_liquidity'] = _current_state_[_tick_upper_]['net_liquidity'] + L
            _current_state_[_tick_upper_]['gross_liquidity'] = _current_state_[_tick_upper_]['gross_liquidity'] - L

            # Close any ticks that have gross_liquidity of zero
            if _current_state_[_tick_lower_]['gross_liquidity'] == 0 and _current_state_[_tick_lower_]['net_liquidity']:
                del _current_state_[_tick_lower_]

            if _current_state_[_tick_upper_]['gross_liquidity'] == 0 and _current_state_[_tick_upper_]['net_liquidity']:
                del _current_state_[_tick_upper_]

            # Remove this liquidity from dataframe
            _lp_dataframe_.loc[len(_lp_dataframe_)] = [_address_, _tick_lower_, _tick_upper_, -L]
            _lp_dataframe_ = _lp_dataframe_.groupby(['address','tick_lower','tick_upper']).sum().reset_index()

            print("System Message: Trader withdraws", str(_token0_amount_), "token0 and", str(_token1_amount_), "token1 from the pool")

            """
            Returns
            """
            # Lastly return the new _current_state_
            return _current_state_, _current_spot_price_, _lp_dataframe_


### Swap 0 to 1

This function is used to swap token 0 to token 1, specifying the amount of token 0.

In [10]:
"""
Parameters:
_current_state_ (dict): this gives the current state (before the trade) of the pool
_current_spot_price_ (float): this gives the current spot price of the pool
_lp_dataframe_ (pandas dataframe): this gives the current LPs of the pool
_fees_collected_ (dict): this gives the fees collected by each LP
_token0_amount_ (float): this gives the amount of token 0 for the trade
_fees_ (float): this is the fee charged by the pool

Results:
_current_state_ (dict): this gives the new state (after the trade) of the pool
_current_spot_price_ (float): this is the new spot price of the pool
_fees_collected_ (dict): this gives the new fees collected by each LP
"""

def Swap_Token0_to_Token1(_current_state_, _current_spot_price_, _lp_dataframe_, _fees_collected_,
                          _token0_amount_, _fees_):
    
    # Initial calculations
    _current_spot_price_ = Decimal(str(_current_spot_price_))
    current_tick = Get_Tick(_current_spot_price_)
    closest_lower_initiated_ticks = np.max([x for x in list(_current_state_.keys()) if x <= current_tick])
    closest_upper_initiated_ticks = np.min([x for x in list(_current_state_.keys()) if x > current_tick])
    price_lower = Get_Price(closest_upper_initiated_ticks)
    price_upper = Get_Price(closest_lower_initiated_ticks)
    current_liquidity = Get_Total_Liquidity(_current_state_, closest_lower_initiated_ticks)
    
    real_balance_token0 = current_liquidity/(_current_spot_price_.sqrt()) - current_liquidity/(price_upper.sqrt())
    real_balance_token1 = current_liquidity*(_current_spot_price_.sqrt()) - current_liquidity*(price_lower.sqrt())

    max_token0_trade_in_this_range = current_liquidity/(price_lower.sqrt()) - current_liquidity/(price_upper.sqrt())

    # Check whether if this swap can be executed within the current initated tick range
    if (1-_fees_)*_token0_amount_ <= max_token0_trade_in_this_range - real_balance_token0:

        """
        If trade CAN be completed in current interval
        """
        
        # Calculate fees earned by each LP
        fees_percentage = _lp_dataframe_[(_lp_dataframe_['tick_lower'] <= closest_lower_initiated_ticks) & (_lp_dataframe_['tick_upper'] >= closest_upper_initiated_ticks)][['address','L']]
        fees_percentage['L'] = fees_percentage['L']/fees_percentage['L'].sum()
        fees_percentage['L'] = fees_percentage['L']*_fees_*_token0_amount_
        for i in np.arange(len(fees_percentage)):
            _fees_collected_[fees_percentage['address'].iloc[i]][0] = _fees_collected_[fees_percentage['address'].iloc[i]][0] + fees_percentage['L'].iloc[i]

        # Calculate new amount of token 0
        real_balance_token0 = real_balance_token0 + (1-_fees_)*_token0_amount_
        # Solve for new amount of token 1
        token1_to_trader = real_balance_token1 - ((current_liquidity**2)/(real_balance_token0 + current_liquidity/(price_upper.sqrt())) - current_liquidity*(price_lower.sqrt()))
        real_balance_token1 = real_balance_token1 - token1_to_trader
        print("System Message: Transfer " + str(token1_to_trader) + " of token1 to trader")

        _new_spot_price_ = (real_balance_token1 + current_liquidity*(price_lower.sqrt()))/(real_balance_token0 + current_liquidity/(price_upper.sqrt()))

        return _current_state_, _new_spot_price_, _fees_collected_, token1_to_trader
    
    else:

        """
        If trade CANNOT be completed in current interval
        """
        
        # This variable records the total amount of token 1 that will be sent to the trader
        total_token1_to_trader = 0

        # Do the same thing until _token0_amount_ is can finish executing in a final range of initiated ticks
        while (1-_fees_)*_token0_amount_ > max_token0_trade_in_this_range - real_balance_token0:

            # Calculate fees earned by each LP
            fees_percentage = _lp_dataframe_[(_lp_dataframe_['tick_lower'] <= closest_lower_initiated_ticks) & (_lp_dataframe_['tick_upper'] >= closest_upper_initiated_ticks)][['address','L']]
            fees_percentage['L'] = fees_percentage['L']/fees_percentage['L'].sum()
            fees_percentage['L'] = fees_percentage['L']*_fees_*(max_token0_trade_in_this_range - real_balance_token0)
            for i in np.arange(len(fees_percentage)):
                _fees_collected_[fees_percentage['address'].iloc[i]][0] = _fees_collected_[fees_percentage['address'].iloc[i]][0] + fees_percentage['L'].iloc[i]

            # Update _token0_amount_
            _token0_amount_ = _token0_amount_ - (1/(1-_fees_))*(max_token0_trade_in_this_range - real_balance_token0)
            
            # First, we do the swap for the current range where the spot price lies
            real_balance_token0 =  max_token0_trade_in_this_range
            token1_to_trader = real_balance_token1 - ((current_liquidity**2)/(real_balance_token0 + current_liquidity/(price_upper.sqrt())) - current_liquidity*(price_lower.sqrt()))
            real_balance_token1 = real_balance_token1 - token1_to_trader

            # Save the amount of token 0 that is swapped within this range
            total_token1_to_trader = total_token1_to_trader + token1_to_trader

            # Update environmental variables
            closest_lower_initiated_ticks = closest_upper_initiated_ticks
            closest_upper_initiated_ticks = np.min([x for x in list(_current_state_.keys()) if x > closest_lower_initiated_ticks])
            price_lower = Get_Price(closest_upper_initiated_ticks)
            price_upper = Get_Price(closest_lower_initiated_ticks)
            current_spot_price = Get_Price(closest_lower_initiated_ticks)
            current_spot_price = Decimal(str(current_spot_price))
            current_liquidity = Get_Total_Liquidity(_current_state_, closest_lower_initiated_ticks)
            real_balance_token0 = current_liquidity/(current_spot_price.sqrt()) - current_liquidity/(price_upper.sqrt())
            real_balance_token1 = current_liquidity*(current_spot_price.sqrt()) - current_liquidity*(price_lower.sqrt())
            max_token0_trade_in_this_range = current_liquidity/(price_lower.sqrt()) - current_liquidity/(price_upper.sqrt())

        # At the last range of initiated where the swap can be completed
        fees_percentage = _lp_dataframe_[(_lp_dataframe_['tick_lower'] <= closest_lower_initiated_ticks) & (_lp_dataframe_['tick_upper'] >= closest_upper_initiated_ticks)][['address','L']]
        fees_percentage['L'] = fees_percentage['L']/fees_percentage['L'].sum()
        fees_percentage['L'] = fees_percentage['L']*_fees_*_token0_amount_
        for i in np.arange(len(fees_percentage)):
            _fees_collected_[fees_percentage['address'].iloc[i]][0] = _fees_collected_[fees_percentage['address'].iloc[i]][0] + fees_percentage['L'].iloc[i]

        real_balance_token0 = real_balance_token0 + (1-_fees_)*_token0_amount_
        token1_to_trader = real_balance_token1 - ((current_liquidity**2)/(real_balance_token0 + current_liquidity/(price_upper.sqrt())) - current_liquidity*(price_lower.sqrt()))
        real_balance_token1 = real_balance_token1 - token1_to_trader

        total_token1_to_trader = total_token1_to_trader + token1_to_trader
        print("System Message: Transfer " + str(total_token1_to_trader) + " of token1 to trader")

        _new_spot_price_ = (real_balance_token1 + current_liquidity*(price_lower.sqrt()))/(real_balance_token0 + current_liquidity/(price_upper.sqrt()))

        return _current_state_, _new_spot_price_, _fees_collected_, total_token1_to_trader
    


### Swap 1 to 0

This function is used to swap token 1 to token 0, specifying the amount of token 1.

In [11]:
"""
Parameters:
_current_state_ (dict): this gives the current state (before the trade) of the pool
_current_spot_price_ (float): this gives the current spot price of the pool
_lp_dataframe_ (pandas dataframe): this gives the current LPs of the pool
_fees_collected_ (dict): this gives the fees collected by each LP
_token0_amount_ (float): this gives the amount of token 0 for the trade
_fees_ (float): this is the fee charged by the pool

Results:
_current_state_ (dict): this gives the new state (after the trade) of the pool
_current_spot_price_ (float): this is the new spot price of the pool
_fees_collected_ (dict): this gives the new fees collected by each LP
"""

def Swap_Token1_to_Token0(_current_state_, _current_spot_price_, _lp_dataframe_, _fees_collected_,
                          _token1_amount_, _fees_):
    
    # Initial calculations
    _current_spot_price_ = Decimal(str(_current_spot_price_))
    current_tick = Get_Tick(_current_spot_price_)
    closest_lower_initiated_ticks = np.max([x for x in list(_current_state_.keys()) if x < current_tick])
    closest_upper_initiated_ticks = np.min([x for x in list(_current_state_.keys()) if x >= current_tick])
    price_lower = Get_Price(closest_upper_initiated_ticks)
    price_upper = Get_Price(closest_lower_initiated_ticks)
    current_liquidity = Get_Total_Liquidity(_current_state_, closest_lower_initiated_ticks)
    
    real_balance_token0 = current_liquidity/(_current_spot_price_.sqrt()) - current_liquidity/(price_upper.sqrt())
    real_balance_token1 = current_liquidity*(_current_spot_price_.sqrt()) - current_liquidity*(price_lower.sqrt())

    max_token1_trade_in_this_range = current_liquidity*(price_upper.sqrt()) - current_liquidity*(price_lower.sqrt())

    # Check whether if this swap can be executed within the current initated tick range
    if (1-_fees_)*_token1_amount_ <= max_token1_trade_in_this_range - real_balance_token1:

        """
        If trade CAN be completed in current interval
        """

        # Calculate fees earned by each LP
        fees_percentage = _lp_dataframe_[(_lp_dataframe_['tick_lower'] <= closest_lower_initiated_ticks) & (_lp_dataframe_['tick_upper'] >= closest_upper_initiated_ticks)][['address','L']]
        fees_percentage['L'] = fees_percentage['L']/fees_percentage['L'].sum()
        fees_percentage['L'] = fees_percentage['L']*_fees_*_token1_amount_
        for i in np.arange(len(fees_percentage)):
            _fees_collected_[fees_percentage['address'].iloc[i]][1] = _fees_collected_[fees_percentage['address'].iloc[i]][1] + fees_percentage['L'].iloc[i]
        
        # Calculate new amount of token 1
        real_balance_token1 = real_balance_token1 + (1-_fees_)*_token1_amount_
        # Solve for new amount of token 0
        token0_to_trader = real_balance_token0 - ((current_liquidity**2)/(real_balance_token1 + current_liquidity*(price_lower.sqrt())) - current_liquidity/(price_upper.sqrt()))
        real_balance_token0 = real_balance_token0 - token0_to_trader
        print("System Message: Transfer " + str(token0_to_trader) + " of token0 to trader")

        _new_spot_price_ = (real_balance_token1 + current_liquidity*(price_lower.sqrt()))/(real_balance_token0 + current_liquidity/(price_upper.sqrt()))

        return _current_state_, _new_spot_price_, _fees_collected_, token0_to_trader
    
    else:

        """
        If trade CANNOT be completed in current interval
        """
        
        # This variable records the total amount of token 0 that will be sent to the trader
        total_token0_to_trader = 0

        # Do the same thing until _token1_amount_ can finish executing in a final range of initiated ticks
        while (1-_fees_)*_token1_amount_ > max_token1_trade_in_this_range - real_balance_token1:

            # Calculate fees earned by each LP
            fees_percentage = _lp_dataframe_[(_lp_dataframe_['tick_lower'] <= closest_lower_initiated_ticks) & (_lp_dataframe_['tick_upper'] >= closest_upper_initiated_ticks)][['address','L']]
            fees_percentage['L'] = fees_percentage['L']/fees_percentage['L'].sum()
            fees_percentage['L'] = fees_percentage['L']*_fees_*(max_token1_trade_in_this_range - real_balance_token1)
            for i in np.arange(len(fees_percentage)):
                _fees_collected_[fees_percentage['address'].iloc[i]][1] = _fees_collected_[fees_percentage['address'].iloc[i]][1] + fees_percentage['L'].iloc[i]

            # Update _token1_amount_
            _token1_amount_ = _token1_amount_ - (1/(1-_fees_))*(max_token1_trade_in_this_range - real_balance_token1)
            
            # First, we do the swap for the current range where the spot price lies
            real_balance_token1 =  max_token1_trade_in_this_range
            token0_to_trader = real_balance_token0 - ((current_liquidity**2)/(real_balance_token1 + current_liquidity*(price_lower.sqrt())) - current_liquidity/(price_upper.sqrt()))
            real_balance_token0 = real_balance_token0 - token0_to_trader

            # Save the amount of token 0 that is swapped within this range
            total_token0_to_trader = total_token0_to_trader + token0_to_trader

            # Update environmental variables
            closest_upper_initiated_ticks = closest_lower_initiated_ticks
            closest_lower_initiated_ticks = np.max([x for x in list(_current_state_.keys()) if x < closest_upper_initiated_ticks])
            price_lower = Get_Price(closest_upper_initiated_ticks)
            price_upper = Get_Price(closest_lower_initiated_ticks)
            current_spot_price = Get_Price(closest_upper_initiated_ticks)
            current_spot_price = Decimal(str(current_spot_price))
            current_liquidity = Get_Total_Liquidity(_current_state_, closest_lower_initiated_ticks)
            real_balance_token0 = current_liquidity/(current_spot_price.sqrt()) - current_liquidity/(price_upper.sqrt())
            real_balance_token1 = current_liquidity*(current_spot_price.sqrt()) - current_liquidity*(price_lower.sqrt())
            max_token1_trade_in_this_range = current_liquidity*(price_upper.sqrt()) - current_liquidity*(price_lower.sqrt())

        # At the last range of initiated where the swap can be completed
        fees_percentage = _lp_dataframe_[(_lp_dataframe_['tick_lower'] <= closest_lower_initiated_ticks) & (_lp_dataframe_['tick_upper'] >= closest_upper_initiated_ticks)][['address','L']]
        fees_percentage['L'] = fees_percentage['L']/fees_percentage['L'].sum()
        fees_percentage['L'] = fees_percentage['L']*_fees_*_token1_amount_
        for i in np.arange(len(fees_percentage)):
            _fees_collected_[fees_percentage['address'].iloc[i]][1] = _fees_collected_[fees_percentage['address'].iloc[i]][1] + fees_percentage['L'].iloc[i]

        real_balance_token1 = real_balance_token1 + (1-_fees_)*_token1_amount_
        token0_to_trader = real_balance_token0 - ((current_liquidity**2)/(real_balance_token1 + current_liquidity*(price_lower.sqrt())) - current_liquidity/(price_upper.sqrt()))
        real_balance_token0 = real_balance_token0 - token0_to_trader

        total_token0_to_trader = total_token0_to_trader + token0_to_trader
        print("System Message: Transfer " + str(total_token0_to_trader) + " of token0 to trader")

        _new_spot_price_ = (real_balance_token1 + current_liquidity*(price_lower.sqrt()))/(real_balance_token0 + current_liquidity/(price_upper.sqrt()))

        return _current_state_, _new_spot_price_, _fees_collected_, total_token0_to_trader


